In [17]:
# Import everything :)
import re
import numpy as np
import pandas as pd 
import os 
import matplotlib.pyplot as plt
#all executions to occur within a cell
%matplotlib inline 
from bs4 import BeautifulSoup
import requests
import seaborn as sns

headers = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36'}
url = "http://www.wisdompanel.com/breed_count_matters/breedlisting/?browseby=size&F_All=Y"

html = requests.get(url, headers=headers).text

# Create a BeautifulSoup document
soup = BeautifulSoup(html, 'html.parser')

# Extract links to all breeds with BeautifulSoup
breed_links = soup.select('h3.itemTitle a')

breed_links = ["http://www.wisdompanel.com/" + a['href'] for a in breed_links]
print("Retrieving data for {} breeds.".format(len(breed_links)))

breed_data = []
for breed in breed_links:

    try:
        breed_html = requests.get(breed, headers=headers).text

        # Use regular expressions to extract name, height, width, description, and group
        name = re.search('<h3>AKC Name:</h3>(.*?)<', breed_html, re.DOTALL).group(1)
        min_weight, max_weight = re.search('<h3>Observed Weight:</h3>(\d+)[\s-]+(\d+)', breed_html, re.DOTALL).groups()
        height = re.search('Average Height:\s+(\d+)', breed_html, re.DOTALL).group(1)
        group = re.search('<h3>Genetic Group:</h3>(.*?)<', breed_html, re.DOTALL).group(1)
        breed_row = [name, min_weight, max_weight, height, group]
        print("Retrieved breed data: ", breed_row)
        breed_data.append(breed_row)
        
    # Some breed pages have formatting errors, so we'll just skip them
    except:
        pass

# Put the data in a pandas DataFrame
breed_data = pd.DataFrame(breed_data, columns=['breed_name', 'min_weight', 'max_weight', 'height', 'group'])

# Create output directory if it doesn't exist
output_dir = os.path.join('..', 'data')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the breed data locally for re-use
output_file = os.path.join(output_dir, 'breed_data.csv')
breed_data.to_csv(output_file, index=False, encoding='utf8'),
print("Successfully saved breed data to {}.".format(output_file))

Retrieving data for 210 breeds.
Successfully saved breed data to ../data/breed_data.csv.
